In [31]:
import pandas as pd
import numpy as np

import nltk
import spacy
from nltk import word_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from gensim.corpora.dictionary import Dictionary
from gensim.models.tfidfmodel import TfidfModel
from gensim.models.ldamodel import LdaModel

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import spacy
from spacy.lemmatizer import Lemmatizer
from spacy.lang.en.stop_words import STOP_WORDS

from tqdm import tqdm_notebook as tqdm
from pprint import pprint

import itertools
from collections import Counter
from collections import defaultdict
from googletrans import Translator
import json
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

In [14]:
df1 = pd.read_csv('tokyo_sept.csv')

/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (61,62) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [15]:
df2 = pd.read_pickle('processed_data.pkl')

In [16]:
df2.head()

,listing_url,id,host_response_time,host_response_rate,host_is_superhost,host_neighbourhood,neighbourhood_cleansed,latitude,longitude,property_type,...,Free parking on premises,Wide hallways,Other pet(s),Host greets you,lat_long,palace_dist,ginza_dist,temple_dist,park_dist,skytree_dist
0,https://www.airbnb.com/rooms/197677,197677,within a day,100.0,1,Sumida District,Sumida Ku,35.71721,139.82596,Apartment,...,0,0,0,0,"(35.717209999999994, 139.82596)",7513.579496,7417.810752,2661.084760,4696.409605,1590.312028
1,https://www.airbnb.com/rooms/776070,776070,within an hour,100.0,1,Kita District,Arakawa Ku,35.73818,139.77009,House,...,0,0,0,0,"(35.73818, 139.77008999999998)",6082.978847,7438.766049,3539.136814,2542.453677,4817.368015
2,https://www.airbnb.com/rooms/905944,905944,within an hour,95.0,1,Shibuya District,Shibuya Ku,35.67968,139.67949,Apartment,...,0,0,0,0,"(35.67968, 139.67949)",6664.499962,7933.153958,11301.364119,9440.211966,12346.096121
3,https://www.airbnb.com/rooms/1016831,1016831,within a few hours,100.0,1,Shimokitazawa,Setagaya Ku,35.65833,139.67153,House,...,0,0,0,0,"(35.65833, 139.67153000000002)",7938.897330,8716.697715,12947.277243,11244.471333,13845.629774
4,https://www.airbnb.com/rooms/1096292,1096292,within a day,100.0,1,Shinjuku District,Shinjuku Ku,35.69098,139.70618,House,...,0,0,0,1,"(35.690979999999996, 139.70618000000002)",4268.373312,5885.015482,8607.893109,6721.937428,9694.372231


In [17]:
df = pd.read_csv('reviews.csv')

In [18]:
df.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,35303,810980,2011-12-28,1502908,Firuz,Miyuki's has been a great host. Helpful and ve...
1,35303,2467161,2012-10-01,350719,Jordan,Nice apartment in one of the best areas of Tok...
2,35303,3572628,2013-02-18,4917704,Aymeric,A very nice host and place. The location is in...
3,35303,3952097,2013-03-30,3243253,Blandine,Miyuki is the most smily face I met when I lan...
4,35303,4377509,2013-05-01,1536097,Kayleigh,The location's wonderful. It's close to Haraju...


In [26]:
subset = df1.loc[df2.index][['id', 'listing_url', 'summary', 'description']]

In [28]:
subset.head()

,id,listing_url,summary,description
0,35303,https://www.airbnb.com/rooms/35303,This shared flat is locating at very close to ...,This shared flat is locating at very close to ...
1,197677,https://www.airbnb.com/rooms/197677,NaN,"We are happy to welcome you to our apartment, ..."
2,289597,https://www.airbnb.com/rooms/289597,NaN,::::::::::::::::::::::::::::::::::::::::::::::...
3,370759,https://www.airbnb.com/rooms/370759,"So close to busy centers, yet so peaceful! Jus...","So close to busy centers, yet so peaceful! Jus..."
4,700253,https://www.airbnb.com/rooms/700253,NaN,::::::::::::::::::::::::::::::::::::::::::::::...


In [32]:
sp = spacy.load('en')

stop_list = ['room', 'bed', 'apartment', 'house', '★', 'bedroom', "  ", '=', '1', '2', '3', '4', '5', '+', 'tokyo']

sp.Defaults.stop_words.update(stop_list)

In [33]:
for word in STOP_WORDS:
    lexeme = sp.vocab[word]
    lexeme.is_stop = True

In [34]:
def lemmatizer(doc):
    doc = [token.lemma_ for token in doc if token.lemma_ != '-PRON-']
    doc = u' '.join(doc)
    return sp.make_doc(doc)

def remove_stopwords(doc):
    # This will remove stopwords and punctuation.
    # Use token.text to return strings, which we'll need for Gensim.
    doc = [token.text for token in doc if token.is_stop != True and token.is_punct != True]
    return doc

In [35]:
sp.add_pipe(lemmatizer,name='lemmatizer',after='ner')
sp.add_pipe(remove_stopwords, name="stopwords", last=True)

In [37]:
newest_doc = subset['description']

In [39]:
sp(newest_doc.iloc[0])

['share',
 'flat',
 'locate',
 'close',
 'harajuku',
 'omotesando',
 'area',
 'enjoy',
 'harajuku',
 'girl',
 'boy',
 'perfect',
 'place',
 'stay',
 'flat',
 'mate',
 'share',
 'kitchen',
 'bathroom',
 'monthly',
 'rental',
 'flat',
 'share',
 'long',
 'term',
 'residence',
 '   ',
 'hide',
 'airbnb',
 'friendly',
 'mature',
 'party',
 'disturbance',
 '   ',
 'note',
 'currently',
 'long',
 'stayer',
 'flat',
 'ask',
 'availability',
 'reserve',
 'c',
 '7',
 'square',
 'meter',
 'single',
 'contact',
 'book',
 'sure',
 'note',
 'newly',
 'refurbish',
 'furnish',
 'shared',
 'fantastic',
 'central',
 'location',
 'easy',
 'access',
 'city',
 "'s",
 'main',
 'area',
 'great',
 'opportunity',
 'live',
 'prime',
 'location',
 'good',
 'value',
 'money',
 'extremely',
 'safe',
 'pleasant',
 'area',
 'gaienmae',
 'boast',
 'stylish',
 'restaurant',
 'plenty',
 'sport',
 'facility',
 'golf',
 'tennis',
 'stadiums',
 'gym',
 'gree']

In [40]:
doc_list = []
# Iterates through each listing decription.
for doc in tqdm(newest_doc):
    # Passes that description through the pipeline and adds to a new list.
    try:
        pr = sp(str(doc))
        doc_list.append(pr)
    except:
        continue

In [43]:
subset.isna().sum()

id             0
listing_url    0
summary        0
description    0
dtype: int64

In [42]:
subset.dropna(inplace=True)

In [44]:
words = corpora.Dictionary(doc_list)

# Turns each document into a bag of words.
corpus = [words.doc2bow(doc) for doc in doc_list]

In [45]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=words,
                                           num_topics=3, 
                                           random_state=2,
                                           update_every=1,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [46]:
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary=lda_model.id2word)
pyLDAvis.display(vis)

/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))
